In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
reviews = pd.read_csv("D:\ds54\projet\data\datasets_cum.csv")

In [57]:
reviews

,app_id,app_name,review_cum
0,10,Counter-Strike,Ruined my life. Only Counter-Strike game that ...
1,10090,Call of Duty: World at War,great game better than any other cod ive playe...
2,10100,King's Quest Collection,'All right! I'm going to play through them all...
3,10110,Space Quest Collection,Great games. Too bad they're delivered via a m...
4,10150,Prototype,This is one of those games that makes you feel...
5,10180,Call of Duty: Modern Warfare 2,this game is the best call of duty ever Don't ...
6,10220,Postal 3,Awesome weapons and gore!
7,10230,Dead Mountaineer's Hotel,Appallingly awful - spent ages setting things ...
8,102500,Kingdoms of Amalur: Reckoning?,Awesome game! pick this up if you love long lo...
9,102600,Orcs Must Die!,"A fun active tower defence game, each level un..."


In [58]:
import string
sp=string.punctuation
sp

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [59]:
reviews['review_cum']=reviews['review_cum'].str.lower()
#suppression de la ponctuation
reviews['review_cum']=reviews['review_cum'].str.translate(str.maketrans('', '', string.punctuation))
#on copie la dataframe pour une utilisation ultérieure
data=reviews.copy()
data.head(15)

,app_id,app_name,review_cum
0,10,Counter-Strike,ruined my life only counterstrike game that wa...
1,10090,Call of Duty: World at War,great game better than any other cod ive playe...
2,10100,King's Quest Collection,all right im going to play through them all in...
3,10110,Space Quest Collection,great games too bad theyre delivered via a mis...
4,10150,Prototype,this is one of those games that makes you feel...
5,10180,Call of Duty: Modern Warfare 2,this game is the best call of duty ever dont b...
6,10220,Postal 3,awesome weapons and gore
7,10230,Dead Mountaineer's Hotel,appallingly awful spent ages setting things u...
8,102500,Kingdoms of Amalur: Reckoning?,awesome game pick this up if you love long lon...
9,102600,Orcs Must Die!,a fun active tower defence game each level unl...


In [60]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [61]:
stop_words = set(stopwords.words('english')) 

#fonction permettant de mettre les mots en liste et de supprimer les stopwords
def tokenise_and_delete_stop_words (text) :       
    word_tokens = word_tokenize(text) 
    filtered_sentence = [w for w in word_tokens if w not in stop_words] 
    return filtered_sentence

#la fonction est appliquée à toute les lignes 
reviews['review_cum']=reviews['review_cum'].apply(tokenise_and_delete_stop_words)
reviews['review_cum'].head()

0    [ruined, life, counterstrike, game, ever, good...
1    [great, game, better, cod, ive, played, first,...
2    [right, im, going, play, row, starts, kings, q...
3    [great, games, bad, theyre, delivered, via, mi...
4    [one, games, makes, feel, like, bad, thoroughl...
Name: review_cum, dtype: object

In [62]:
tfidf_transformer = TfidfVectorizer(analyzer = tokenise_and_delete_stop_words).fit(data['review_cum'])
tfidf_data = tfidf_transformer.transform(data['review_cum'])
tfidf_data.shape

(12, 410)

In [64]:
cosine_similarities = linear_kernel(tfidf_data, tfidf_data)

In [65]:
cosine_similarities

array([[1.        , 0.236776  , 0.01743275, 0.03501173, 0.11941339,
        0.26620911, 0.02305937, 0.0073925 , 0.17926457, 0.22725636,
        0.21346103, 0.        ],
       [0.236776  , 1.        , 0.01628022, 0.06508   , 0.07354671,
        0.23792314, 0.0208308 , 0.        , 0.1981084 , 0.20113081,
        0.24784874, 0.        ],
       [0.01743275, 0.01628022, 1.        , 0.        , 0.        ,
        0.00740028, 0.        , 0.        , 0.05839597, 0.        ,
        0.01644482, 0.        ],
       [0.03501173, 0.06508   , 0.        , 1.        , 0.08083385,
        0.        , 0.        , 0.02655565, 0.03068066, 0.02012276,
        0.03194782, 0.        ],
       [0.11941339, 0.07354671, 0.        , 0.08083385, 1.        ,
        0.05515751, 0.        , 0.0226001 , 0.13162211, 0.06653828,
        0.05478272, 0.        ],
       [0.26620911, 0.23792314, 0.00740028, 0.        , 0.05515751,
        1.        , 0.        , 0.00916503, 0.16474026, 0.15158189,
        0.22550245,

In [85]:
data1 = pd.DataFrame(cosine_similarities)

In [86]:
data1.to_csv('tf_idf_cos.csv')

In [83]:
def item(id):  
  return reviews.loc[reviews['app_id'] == id]['app_name'].tolist()[0]  #itemid to name
#Recommander les jeux les plus similaires
def recommend(item_id, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id) + "...")   
    print("-------")
    idx = reviews[reviews["app_id"]== item_id].index.tolist()[0] #itemid to index
    similar_indices = cosine_similarities[idx].argsort()[::-1]  #sort the array in ascending order and return the index
    for i in range(num): 
       print("Recommended: " + item(reviews["app_id"][similar_indices[i+1]]) + " (score:" +      str(cosine_similarities[idx][similar_indices[i+1]]) + ")")

In [84]:
recommend(item_id=10, num=2)

Recommending 2 products similar to Counter-Strike...
-------
Recommended: Call of Duty: Modern Warfare 2 (score:0.26620910544981524)
Recommended: Call of Duty: World at War (score:0.23677600184678743)
